In [ ]:
import os, sys
import pandas as pd
import torch


dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
dir0 = os.path.dirname(dir1)

if dir1 not in sys.path: sys.path.append(dir0)

from src.config import PPOConfig, EmbeddingStrategy
from src.experiments import ExperimentSuite
from src.utils import ExperimentUtils

import logging

logger = logging.getLogger('torchrl')
logger.setLevel(logging.WARNING)  # show only WARNING and above

def rewards_of_suite(suite, agents):
    all_df = pd.DataFrame()
    for i in agents:
        change_to_config = {"n_agents": i}
        df = suite.rollout_all_get_rewards(change_to_config)

        # Rename the reward column to a common name and add the number of agents
        reward_col = df.columns[1]  # Assumes second column is the reward
        df = df.rename(columns={reward_col: "rewards"})
        df["agents"] = i

        all_df = pd.concat([all_df, df[["experiment", "agents", "rewards"]]], ignore_index=True)

    all_df = all_df.rename(columns={"experiment": "strategy"})
    return all_df


def run_generalizability(strategies, file_name, training_agents, testing_agents, iters):
    my_device = torch.device("cpu")
    df_all_strategies_rollout = pd.DataFrame()
    df_all_strategies_train = pd.DataFrame()

    url = "saved_experiments" + "/" + file_name + '.csv'
    url_rollout = "saved_experiments" + "/" + file_name + '_rollout' + '.csv'


    for strategy in strategies:
        base_config_balance_5_agents = PPOConfig(
            scenario_name='balance', max_agents=max(testing_agents), use_strategy_defaults=True, n_iters=iters, decentralized_execution=False
        )

        param_grid = {
            "strategy": [strategy],
        }
        suite = ExperimentSuite(base_config=base_config_balance_5_agents, param_grid=param_grid, name="test_all", device=my_device)

        # try:
            # train on training_agents
        for i in training_agents:
            change_to_config = {"n_agents": i}
            suite.create_and_run_experiments_with_updated_config(change_to_config, create_new=False, k=1)

        # test on testing_agents
        df = rewards_of_suite(suite, testing_agents)
        df_all_strategies_rollout = df_all_strategies_rollout.append(df)

        suite_utils = ExperimentUtils(experiment_suite=suite)import os, sys
import pandas as pd
import torch


dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
dir0 = os.path.dirname(dir1)

if dir1 not in sys.path: sys.path.append(dir0)

from src.config import PPOConfig, EmbeddingStrategy
from src.experiments import ExperimentSuite
from src.utils import ExperimentUtils

import logging

logger = logging.getLogger('torchrl')
logger.setLevel(logging.WARNING)  # show only WARNING and above

def rewards_of_suite(suite, agents):
    all_df = pd.DataFrame()
    for i in agents:
        change_to_config = {"n_agents": i}
        df = suite.rollout_all_get_rewards(change_to_config)

        # Rename the reward column to a common name and add the number of agents
        reward_col = df.columns[1]  # Assumes second column is the reward
        df = df.rename(columns={reward_col: "rewards"})
        df["agents"] = i

        all_df = pd.concat([all_df, df[["experiment", "agents", "rewards"]]], ignore_index=True)

    all_df = all_df.rename(columns={"experiment": "strategy"})
    return all_df


def run_generalizability(strategies, file_name, training_agents, testing_agents, iters):
    my_device = torch.device("cpu")
    df_all_strategies_rollout = pd.DataFrame()
    df_all_strategies_train = pd.DataFrame()

    url = "saved_experiments" + "/" + file_name + '.csv'
    url_rollout = "saved_experiments" + "/" + file_name + '_rollout' + '.csv'


    for strategy in strategies:
        base_config_balance_5_agents = PPOConfig(
            scenario_name='balance', max_agents=max(testing_agents), use_strategy_defaults=True, n_iters=iters, decentralized_execution=True
        )

        param_grid = {
            "strategy": [strategy],
        }
        suite = ExperimentSuite(base_config=base_config_balance_5_agents, param_grid=param_grid, name="test_all", device=my_device)

        # try:
            # train on training_agents
        for i in training_agents:
            change_to_config = {"n_agents": i}
            suite.create_and_run_experiments_with_updated_config(change_to_config, create_new=False, k=1)

        # test on testing_agents
        df = rewards_of_suite(suite, testing_agents)
        df_all_strategies_rollout = df_all_strategies_rollout.append(df)

        suite_utils = ExperimentUtils(experiment_suite=suite)
        df_all_strategies_train = df_all_strategies_train.append(suite_utils.df)

        # except Exception as e:
        # print(e)
        print(f"Failed to compute for {strategy}")

        df_all_strategies_rollout.to_csv(url_rollout, index=False)
        df_all_strategies_train.to_csv(url, index=False)
        print('Successful generalizability for ', str(strategy))



        df_all_strategies_train = df_all_strategies_train.append(suite_utils.df)

        # except Exception as e:
        # print(e)
        print(f"Failed to compute for {strategy}")

        df_all_strategies_rollout.to_csv(url_rollout, index=False)
        df_all_strategies_train.to_csv(url, index=False)
        print('Successful generalizability for ', str(strategy))




In [ ]:
file_name='high_variance_ctce'
high_variance = [4, 6, 8, 10, 12, 14, 16, 18]
n_iters = int(8/len(high_variance))
strategies = [
    # EmbeddingStrategy.CONCAT,
    EmbeddingStrategy.MLP,
    EmbeddingStrategy.MLP_LOCAL,
    EmbeddingStrategy.MLP_GLOBAL,
    EmbeddingStrategy.GRAPH_SAGE,
    EmbeddingStrategy.GRAPH_GAT,
    EmbeddingStrategy.GRAPH_GAT_v2,
    EmbeddingStrategy.SET_TRANSFORMER_INV,
    EmbeddingStrategy.TRANSFORMER_GLOBAL,
]

run_generalizability(strategies, file_name=file_name, training_agents=high_variance, testing_agents=[5, 10, 20], iters=n_iters)

In [ ]:
file_name='low_variance_ctce'
low_variance = [6, 10 , 14, 18]
n_iters = int(80/len(low_variance))
strategies = [
    # EmbeddingStrategy.CONCAT,
    EmbeddingStrategy.MLP,
    EmbeddingStrategy.MLP_LOCAL,
    EmbeddingStrategy.MLP_GLOBAL,
    EmbeddingStrategy.GRAPH_SAGE,
    EmbeddingStrategy.GRAPH_GAT,
    EmbeddingStrategy.GRAPH_GAT_v2,
    EmbeddingStrategy.SET_TRANSFORMER_INV,
    EmbeddingStrategy.TRANSFORMER_GLOBAL,
]

suite, df = run_generalizability(strategies, file_name=file_name, training_agents=low_variance, testing_agents=[5, 10, 20], iters=n_iters)

In [ ]:
file_name='no_variance_ctce'
no_variance = [10]
n_iters = int(80/len(no_variance))
strategies = [
    # EmbeddingStrategy.CONCAT,
    EmbeddingStrategy.MLP,
    EmbeddingStrategy.MLP_LOCAL,
    EmbeddingStrategy.MLP_GLOBAL,
    EmbeddingStrategy.GRAPH_SAGE,
    EmbeddingStrategy.GRAPH_GAT,
    EmbeddingStrategy.GRAPH_GAT_v2,
    EmbeddingStrategy.SET_TRANSFORMER_INV,
    EmbeddingStrategy.TRANSFORMER_GLOBAL,
]

run_generalizability(strategies, file_name=file_name, training_agents=no_variance, testing_agents=[5, 10, 20], iters=n_iters)